In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, sum as _sum, col, sqrt, date_format, concat_ws
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, FloatType, Row
import pyspark.sql.functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from datetime import datetime
import calendar

spark = SparkSession.builder.appName("FundFlowLinearRegressionPrediction").getOrCreate()
schema_full = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("report_date", IntegerType(), False),
    StructField("tBalance", IntegerType(), False),
    StructField("yBalance", IntegerType(), False),
    StructField("total_purchase_amt", IntegerType(), False),
    StructField("direct_purchase_amt", IntegerType(), False),
    StructField("purchase_bal_amt", IntegerType(), False),
    StructField("purchase_bank_amt", IntegerType(), False),
    StructField("total_redeem_amt", IntegerType(), False),
    StructField("consume_amt", IntegerType(), False),
    StructField("transfer_amt", IntegerType(), False),
    StructField("tftobal_amt", IntegerType(), False),
    StructField("tftocard_amt", IntegerType(), False),
    StructField("share_amt", IntegerType(), False),
    StructField("category1", StringType(), True),
    StructField("category2", StringType(), True),
    StructField("category3", StringType(), True),
    StructField("category4", StringType(), True)
])

data_path = "user_balance_table.csv"  
df = spark.read.csv(data_path, header=True, schema=schema_full)
df = df.drop("user_id", "category1", "category2", "category3", "category4")
def int_to_date(date_int):
    return datetime.strptime(str(date_int), "%Y%m%d")

int_to_date_udf = udf(int_to_date, DateType())
df = df.withColumn("report_date", int_to_date_udf("report_date"))
aggregated_df = df.groupBy("report_date").agg(_sum("total_purchase_amt").alias("total_purchase_amt"),_sum("total_redeem_amt").alias("total_redeem_amt"))
aggregated_df = aggregated_df.withColumn("year", F.year("report_date")) \
                             .withColumn("month", F.month("report_date")) \
                             .withColumn("day", F.dayofmonth("report_date")) \
                             .withColumn("day_of_week", F.dayofweek("report_date"))

feature_cols = ["year", "month", "day", "day_of_week"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
assembled_df = assembler.transform(aggregated_df)
assembled_df.select("report_date", "features", "total_purchase_amt", "total_redeem_amt").show(5)
train_df = assembled_df.filter(F.col("report_date") <= '2014-08-31')
year = 2014
month = 9 
_, num_days = calendar.monthrange(year, month) 
prediction_dates = [datetime.strptime(f"{year}-{month:02d}-{day:02d}", "%Y-%m-%d") for day in range(1, num_days + 1)]
from pyspark.ml.regression import LinearRegression

def recursive_linear_regression_prediction(spark, train_df, assembler, prediction_dates):
    predictions = []
    lr_purchase = LinearRegression(featuresCol='features', labelCol='total_purchase_amt')
    lr_redeem = LinearRegression(featuresCol='features', labelCol='total_redeem_amt')
    model_purchase = lr_purchase.fit(train_df)
    model_redeem = lr_redeem.fit(train_df)
    
    for current_date in prediction_dates:
        print(f"{current_date.strftime('%Y-%m-%d')}")
        year_val = current_date.year
        month_val = current_date.month
        day_val = current_date.day
        day_of_week_val = current_date.isoweekday()
        new_row_data = [(year_val, month_val, day_val, day_of_week_val)]
        new_row_schema = StructType([
            StructField("year", IntegerType(), False),
            StructField("month", IntegerType(), False),
            StructField("day", IntegerType(), False),
            StructField("day_of_week", IntegerType(), False)
        ])
        
        new_row = spark.createDataFrame(new_row_data, schema=new_row_schema)
        assembled_new_row = assembler.transform(new_row)
        purchase_pred = model_purchase.transform(assembled_new_row).select("prediction").collect()[0][0]
        redeem_pred = model_redeem.transform(assembled_new_row).select("prediction").collect()[0][0]
        purchase_pred = max(0, int(round(purchase_pred)))
        redeem_pred = max(0, int(round(redeem_pred)))
        predictions.append({
            "report_date": current_date.strftime("%Y%m%d"),  
            "predicted_total_purchase_amt": purchase_pred,
            "predicted_total_redeem_amt": redeem_pred
        })
        new_train_row_data = [(current_date, year_val, month_val, day_val, day_of_week_val, purchase_pred, redeem_pred)]
        new_train_schema = StructType([
            StructField("report_date", DateType(), False),
            StructField("year", IntegerType(), False),
            StructField("month", IntegerType(), False),
            StructField("day", IntegerType(), False),
            StructField("day_of_week", IntegerType(), False),
            StructField("total_purchase_amt", IntegerType(), False),
            StructField("total_redeem_amt", IntegerType(), False)
        ])
        
        new_train_row = spark.createDataFrame(new_train_row_data, schema=new_train_schema)
        assembled_new_train_row = assembler.transform(new_train_row)
        train_df = train_df.union(assembled_new_train_row)
        model_purchase = lr_purchase.fit(train_df)
        model_redeem = lr_redeem.fit(train_df)
    predictions_rows = [Row(**pred) for pred in predictions]
    predictions_df = spark.createDataFrame(predictions_rows)
    
    return predictions_df
predictions_df = recursive_linear_regression_prediction(
    spark=spark,
    train_df=train_df,
    assembler=assembler,
    prediction_dates=prediction_dates
)
predictions_df.show(10)

24/12/08 17:04:40 WARN Utils: Your hostname, jyxiedeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.22.77.245 instead (on interface en0)
24/12/08 17:04:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/08 17:04:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/08 17:04:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


root
 |-- report_date: integer (nullable = true)
 |-- tBalance: integer (nullable = true)
 |-- yBalance: integer (nullable = true)
 |-- total_purchase_amt: integer (nullable = true)
 |-- direct_purchase_amt: integer (nullable = true)
 |-- purchase_bal_amt: integer (nullable = true)
 |-- purchase_bank_amt: integer (nullable = true)
 |-- total_redeem_amt: integer (nullable = true)
 |-- consume_amt: integer (nullable = true)
 |-- transfer_amt: integer (nullable = true)
 |-- tftobal_amt: integer (nullable = true)
 |-- tftocard_amt: integer (nullable = true)
 |-- share_amt: integer (nullable = true)



+-----------+--------+--------+------------------+-------------------+----------------+-----------------+----------------+-----------+------------+-----------+------------+---------+
|report_date|tBalance|yBalance|total_purchase_amt|direct_purchase_amt|purchase_bal_amt|purchase_bank_amt|total_redeem_amt|consume_amt|transfer_amt|tftobal_amt|tftocard_amt|share_amt|
+-----------+--------+--------+------------------+-------------------+----------------+-----------------+----------------+-----------+------------+-----------+------------+---------+
|   20140805|   20385|   20383|                 2|                  0|               0|                0|               0|          0|           0|          0|           0|        2|
|   20140808|   20391|   20389|                 2|                  0|               0|                0|               0|          0|           0|          0|           0|        2|
|   20140811|   20397|   20395|                 2|                  0|               

+-----------+--------+--------+------------------+-------------------+----------------+-----------------+----------------+-----------+------------+-----------+------------+---------+
|report_date|tBalance|yBalance|total_purchase_amt|direct_purchase_amt|purchase_bal_amt|purchase_bank_amt|total_redeem_amt|consume_amt|transfer_amt|tftobal_amt|tftocard_amt|share_amt|
+-----------+--------+--------+------------------+-------------------+----------------+-----------------+----------------+-----------+------------+-----------+------------+---------+
| 2014-08-05|   20385|   20383|                 2|                  0|               0|                0|               0|          0|           0|          0|           0|        2|
| 2014-08-08|   20391|   20389|                 2|                  0|               0|                0|               0|          0|           0|          0|           0|        2|
| 2014-08-11|   20397|   20395|                 2|                  0|               

+-----------+------------------+----------------+
|report_date|total_purchase_amt|total_redeem_amt|
+-----------+------------------+----------------+
| 2013-09-09|          49312473|        45621186|
| 2013-09-19|          24778048|        11418512|
| 2014-08-01|         374884735|       252540858|
| 2014-06-03|         270887462|       385622582|
| 2014-06-11|         327661453|       246127540|
| 2014-02-16|         331324628|       183057224|
| 2014-01-24|         614103894|       345829001|
| 2013-09-12|          68573684|        45147220|
| 2014-08-13|         261506619|       303975517|
| 2013-11-18|         195766477|       136916782|
+-----------+------------------+----------------+
only showing top 10 rows



...

预测日期: 2014-09-29
预测 total_purchase_amt: 222246394, total_redeem_amt: 265909436


24/12/08 17:17:45 WARN Instrumentation: [e9f4823c] regParam is zero, which might cause numerical instability and overfitting.
24/12/08 17:17:45 WARN DAGScheduler: Broadcasting large task binary with size 1038.8 KiB
24/12/08 17:17:49 WARN DAGScheduler: Broadcasting large task binary with size 1039.8 KiB
24/12/08 17:18:03 WARN Instrumentation: [23c7b4f5] regParam is zero, which might cause numerical instability and overfitting.
24/12/08 17:18:03 WARN DAGScheduler: Broadcasting large task binary with size 1038.7 KiB
24/12/08 17:18:08 WARN DAGScheduler: Broadcasting large task binary with size 1039.8 KiB


预测日期: 2014-09-30
预测 total_purchase_amt: 212374558, total_redeem_amt: 256383115


24/12/08 17:18:23 WARN Instrumentation: [962ffddd] regParam is zero, which might cause numerical instability and overfitting.
24/12/08 17:18:23 WARN DAGScheduler: Broadcasting large task binary with size 1071.0 KiB
24/12/08 17:18:27 WARN DAGScheduler: Broadcasting large task binary with size 1072.1 KiB
24/12/08 17:18:42 WARN Instrumentation: [e804b238] regParam is zero, which might cause numerical instability and overfitting.
24/12/08 17:18:42 WARN DAGScheduler: Broadcasting large task binary with size 1071.0 KiB
24/12/08 17:18:47 WARN DAGScheduler: Broadcasting large task binary with size 1072.1 KiB


+-----------+----------------------------+--------------------------+
|report_date|predicted_total_purchase_amt|predicted_total_redeem_amt|
+-----------+----------------------------+--------------------------+
|   20140901|                   286724349|                 301312857|
|   20140902|                   273757046|                 289312134|
|   20140903|                   262649261|                 279232117|
|   20140904|                   252716229|                 270362950|
|   20140905|                   243388881|                 262107808|
|   20140906|                   234184023|                 253952513|
|   20140907|                   224691093|                 245452428|
|   20140908|                   260706319|                 283168628|
|   20140909|                   250616507|                 274010454|
|   20140910|                   241884362|                 266299884|
+-----------+----------------------------+--------------------------+
only showing top 10 

In [2]:
from pyspark.sql.functions import concat_ws
formatted_predictions_df = predictions_df.select(
    concat_ws(",", 
              col("report_date"),
              col("predicted_total_purchase_amt"),
              col("predicted_total_redeem_amt")
             ).alias("prediction")
)

formatted_predictions_df.coalesce(1).write.mode("overwrite").text("task3_result")